# Getting started

This notebook showcases the main functionalities of tri3d.
It can be run for any of the supported dataset.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import k3d
import matplotlib.pyplot as plt
import numpy as np

from tri3d.datasets import KITTIObject, NuScenes, Once, Waymo, ZODFrames
from tri3d.plot import (
    plot_bbox_cam,
    plot_annotations_bev,
    plot_bboxes_3d,
    to_k3d_colors,
    gen_discrete_cmap,
)
from tri3d.geometry import test_box_in_frame

Uncomment the desired dataset below to visualize another dataset.
The rest of the notebook should run as-is thanks to the shared API.

In [ ]:
# dataset = KITTIObject("../../datasets/kittiobject")
# camera, image, lidar = "cam", "img2", "velo"

# dataset = NuScenes("../../datasets/nuscenes", "v1.0-mini")
# camera, image, lidar = "CAM_FRONT", "IMG_FRONT", "LIDAR_TOP"

# dataset = Once("../../datasets/once", "train")
# camera, image, lidar = "cam01", "img01", "lidar_roof"

dataset = Waymo("../../datasets/waymo", split="training")
camera, image, lidar = "CAM_FRONT", "IMG_FRONT", "LIDAR_TOP"

# dataset = ZODFrames(
#     "../../datasets/zodframes",
#     "trainval-frames-mini.json",
#     "train",
# )
# camera, image, lidar = "front", "img_front", "velodyne"

## Pose and calibration

In order to plot sensor poses, we first use `Dataset.cam_sensors` and `Dataset.pcl_sensors` to get the sensor names.
The list of available frames for these sensors is given by `Dataset.frames()`.

Using `Dataset.poses()` returns the poses of a given sensor at every sample frame available for that sensor.
Poses are expressed in an arbitrary *world* coordinate system which is defined per sequence.

A pose is actually the geometric transform which takes a point in the sensor-local coordinates and returns it in *world* woordinates.
In tri3d, poses are stored as instances of `tri3d.geometry.Transform`, which contain a batch of transformations (one for each frame).
The geometric transformations objects support inversion, composition and, of course, application to 3D points.

In [ ]:
plot = k3d.plot(name="pcl", camera_mode="orbit", camera_auto_fit=False)

# select the first lidar pose as reference coordinates
p0 = dataset.poses(seq=0, sensor=lidar)[0]

# iterate over sensors
for sensor in dataset.pcl_sensors + dataset.cam_sensors:
    if len(dataset.timestamps(seq=0, sensor=sensor)) == 0:
        continue

    # get the sensor poses as a batched geometric transformation
    poses = dataset.poses(seq=0, sensor=sensor)

    print(f"{sensor:15s} : {len(poses)} frames")

    # express poses relative to p0 instead of world coordinates
    poses = p0.inv() @ poses

    # Get sensor positions at each frame by projecting [0, 0, 0].
    # Note that [0, 0, 0] is automatically broadcasted to len(poses).
    origins = poses.apply(np.zeros([3]))

    # plot pose axes
    for edge, color in zip(np.eye(3) * 0.2, [0xFF0000, 0x00FF00, 0x0000FF]):
        # get axis vector by projecting [1, 0, 0], [0, 1, 0], [0, 0, 1]
        vectors = poses.apply(edge) - origins

        plot += k3d.vectors(
            origins.astype(np.float32),
            vectors.astype(np.float32),
            color=color,
            head_size=0.3,
            group=sensor,
        )

plot.camera = [6.44, -4.52, 3.66, 11.35, 6.17, -3.55, 0.0, 0.0, 1.0]
plot

## Annotations in frame

Requesting annotations in image coordinates will automatically interpolate and project annotations to the timestamps and coordinates of the camera.

Two helpers are used to reduce the boilerplate: `tri3d.plot.test_box_in_frame` and `tri3d.plot.plot_bbox_cam`.

In [ ]:
# retrieve image
sample_image = dataset.image(seq=0, frame=0, sensor=image)

# retrieve boxes in image coordinates, with trajectories interpolated 
# at the image timestamp if necessary
sample_annotations = dataset.boxes(seq=0, frame=0, coords=image)

In [ ]:
img_size = sample_image.size
cmap = gen_discrete_cmap(len(dataset.det_labels))

plt.figure(dpi=125, figsize=(10, 6))
plt.xlim((0, img_size[0]))
plt.ylim((img_size[1], 0))

plt.imshow(sample_image)

for i, ann in enumerate(sample_annotations):
    # skip boxes where no vertex is in the frame
    if test_box_in_frame(ann.transform, ann.size, img_size):
        obj_id = str(getattr(ann, "instance", i))[:4]
        hdg = ann.heading / np.pi * 180
        color = cmap(dataset.det_labels.index(ann.label))
        u, v, z = ann.center
        print(f"  {obj_id:4s}: {ann.label:20s} at {z:3.0f}m heading {hdg:+.0f}°")

        # plot the bounding box
        plot_bbox_cam(ann.transform, ann.size, img_size, c=color)

        # plot the uid
        plt.text(
            np.clip(u, 0, img_size[0]),
            np.clip(v, 0, img_size[1]),
            obj_id,
            horizontalalignment="center",
            verticalalignment="center",
            bbox=dict(facecolor="white", linewidth=0, alpha=0.7),
        )

## Lidar points over images

Requesting points in image coordinates will automatically compensate for ego car movement between the two sensor timestamps.
It will also apply extrinsic and intrinsic calibrations.

In [ ]:
# retrieve image
sample_image = dataset.image(seq=0, frame=0, sensor=image)

# retrieve points in image coordinates, ego car movement between the lidar and
# camera timestamps is compensated if necessary.
points = dataset.points(seq=0, frame=0, sensor=lidar, coords=image)

# discard extra dims (intensity, etc.)
uvz = points[:, :3]

# only keep points that are visible in the image
in_frame = (
    (uvz[:, 0] > 0)
    & (uvz[:, 1] > 0)
    & (uvz[:, 2] > 0) # in front of camera
    & (uvz[:, 0] < sample_image.size[0])
    & (uvz[:, 1] < sample_image.size[1])
)
uvz = uvz[in_frame]

In [ ]:
plt.figure(dpi=125, figsize=(10, 6))
plt.xlim((0, img_size[0]))
plt.ylim((img_size[1], 0))

plt.imshow(sample_image)
plt.scatter(uvz[:, 0], uvz[:, 1], c=uvz[:, 2], clim=(0, 40), s=0.5, alpha=1)

plt.xlim(0, sample_image.size[0])
plt.ylim(sample_image.size[1], 0)
plt.gca().set_aspect("equal")

## Lidar points and boxes

In [ ]:
# retrieve point cloud
sample_pcl = dataset.points(seq=0, frame=0, coords=lidar, sensor=lidar)

# retrieve boxes in lidar coordinates
sample_annotations = dataset.boxes(seq=0, frame=0, coords=lidar)

In [ ]:
plt.figure(dpi=125, figsize=(8, 8))

plt.scatter(
    sample_pcl[:, 0], sample_pcl[:, 1], s=5, c="k", alpha=0.6, edgecolors="none"
)

cmap = gen_discrete_cmap(len(dataset.det_labels))

plot_annotations_bev(
    [a.center for a in sample_annotations],
    [a.size for a in sample_annotations],
    [a.heading for a in sample_annotations],
    ax=plt.gca(),
    c=cmap([dataset.det_labels.index(a.label) for a in sample_annotations]),
)

plt.xlim(-25, 25)
plt.ylim(-25, 25)
plt.gca().set_aspect("equal")

In [ ]:
xyz = dataset.points(seq=0, frame=0, coords=lidar, sensor=lidar)[:, :3].astype(np.float32)

sample_annotations = dataset.boxes(seq=0, frame=0, coords=lidar)
sample_annotations = [a for a in sample_annotations if a.label != "DontCare"]

c = to_k3d_colors(plt.get_cmap("viridis")((xyz[:, 2] + 2) / 4))

plot = k3d.plot(name="pcl", camera_mode="orbit", camera_auto_fit=False)

plot += k3d.points(positions=xyz, point_size=1, shader="dot", colors=c)

plot_bboxes_3d(
    plot,
    [a.transform for a in sample_annotations],
    [a.size for a in sample_annotations],
    c=0xFF0000,
)

plot.camera = [-0.52, -12.19, 17.0, 4.94, -1.7, 5.19, 0.0, 0.0, 1.0]
plot

In [ ]:
if isinstance(dataset, Waymo):  # waymo does not have segmentation for all frames
    seq, frame = dataset.frames(0, sensor="SEG_LIDAR_TOP")[0]
elif isinstance(dataset, NuScenes):
    seq = 0
    frame = dataset.keyframes(seq, lidar)[0]
else:
    seq, frame = 0, 0

xyz = dataset.points(seq, frame, sensor=lidar)[:, :3].astype(np.float32)
semantic = dataset.semantic(seq, frame, sensor=lidar)

cmap = gen_discrete_cmap(len(dataset.sem_labels))
c = to_k3d_colors(cmap(semantic)) * (semantic >= 0)

plot = k3d.plot(name="pcl", camera_mode="orbit", camera_auto_fit=False)

plot += k3d.points(positions=xyz, point_size=1, shader="dot", colors=c)

plot.camera = [-0.52, -12.19, 17.0, 4.94, -1.7, 5.19, 0.0, 0.0, 1.0]
plot

In [ ]:
if isinstance(dataset, Waymo):  # waymo does not have segmentation for all frames
    seq, frame = dataset.frames(1, sensor="SEG_IMG_FRONT")[0]
else:
    seq, frame = 0, 0

instances = dataset.instances2d(seq, frame, "IMG_FRONT")
segmentation = dataset.semantic2d(seq, frame, "IMG_FRONT")

plt.figure(dpi=125, figsize=(8, 14))

plt.subplot(3, 1, 1)
plt.imshow(dataset.image(seq, frame, image))

plt.subplot(3, 1, 2)
cmap = gen_discrete_cmap(len(dataset.sem2d_labels))
plt.imshow(cmap(segmentation))

plt.subplot(3, 1, 3)
cmap = gen_discrete_cmap(np.max(instances) + 1)
plt.imshow(cmap(instances))